In [1]:
# Basic tools
import os
import sys

# Data tools
import numpy as np
import pandas as pd
import scipy.spatial.distance as sdist
from scipy.stats import norm

# Viz tools
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# NLP tools
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec, KeyedVectors
from gensim.corpora.wikicorpus import WikiCorpus
import spacy
snlp = spacy.load('en_core_web_sm')

# Local
## Allow local relative imports
module_path = os.path.abspath('..')
include_path = os.path.join(module_path, 'include')
data_path = os.path.join(module_path, 'data')
if include_path not in sys.path:
    sys.path.append(include_path)
from my_nlp import Tokenizer

In [12]:
sent_df = pd.read_csv(data_path + '/clickstream-enwiki-2018-08-sentences.tsv', sep = '\t').dropna()

In [5]:
# embed = Word2Vec.load('/Volumes/BlackBoxNew/insight/en_1000_no_stem/en.model')
embed = KeyedVectors.load("/Volumes/BlackBoxNew/insight/en_1000_no_stem/en.model.kv", mmap='r')

In [9]:
def cosine_dist(row):
    u = np.zeros(1000)
    i = 0
    for word in row['origin_sent'].split():
        if word in embed:
            u += embed[word]
            
    u = np.array([embed[x] for x in row['origin_sent'].split() if x in embed]).mean(axis = 0)
    v = np.array([embed[x] for x in row['target_sent'].split() if x in embed]).mean(axis = 0)
        return sdist.cosine(u, v), row['match']
    else:
        return None, row['match']

In [13]:
test_df1 = sent_df[['origin_sent', 'target_sent', 'match']].apply(cosine_dist, axis = 1, result_type = 'expand').rename(columns = {0: 'dist', 1: 'match'})
test_df1.groupby(by = ['match']).mean()

/usr/local/anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: Mean of empty slice.
/usr/local/anaconda3/envs/insight/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: Mean of empty slice.
  app.launch_new_instance()


KeyboardInterrupt: 